In [1]:
import pandas as pd
import spacy
from spacy.tokens import Doc
from spacy.vocab import Vocab
from spacy.training import Example
import random
import torch
import numpy as np
from spacy.util import minibatch, compounding
from spacy import displacy
from spacy import __version__ as spacy_version


In [3]:
# Load data from Excel sheet
df = pd.read_excel('MasterList (3).xlsx')
texts = df['Text'].tolist()
labels = df['Name'].tolist()


In [21]:
# Remove empty strings from text data
texts = [text for text in texts if text]

In [27]:
# Set up blank Spacy model
#nlp = spacy.blank('en')
#vocab = Vocab()
#nlp.vocab = vocab

In [28]:
#vocab

In [29]:
nlp = spacy.load('en_core_web_sm')
nlp.vocab.vectors.name = 'spacy_pretrained_vectors'

In [30]:
# Define your BiLSTM model architecture
class BiLSTMModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(BiLSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim, bidirectional=True)
        self.fc = torch.nn.Linear(hidden_dim * 2, output_dim)
        
    def forward(self, input):
        lstm_out, _ = self.lstm(input.view(len(input), 1, -1))
        out = self.fc(torch.cat((lstm_out[-1,:, :self.hidden_dim], lstm_out[0,:, self.hidden_dim:]), dim=1))
        return out


In [31]:

# Set up your training hyperparameters
epochs = 10
learning_rate = 0.001
batch_size = 8
dropout = 0.2
input_dim = 300
hidden_dim = 128
output_dim = nlp.vocab.vectors_length

# Initialize your model
model = BiLSTMModel(input_dim, hidden_dim, output_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.MSELoss()

In [33]:
# Train your model
for epoch in range(epochs):
    random.seed(epoch)
    losses = {}
    batches = minibatch(zip(texts, labels), size=compounding(batch_size, len(texts), 1.001))
    for batch in batches:
        texts, labels = zip(*batch)
        docs = [nlp(text) for text in texts]
        X = torch.FloatTensor([doc.vector for doc in docs])
        y = torch.FloatTensor([np.asarray(label) for label in labels])
        model.zero_grad()
        preds = model(X)
        loss = loss_fn(preds, y)
        loss.backward()
        optimizer.step()
        losses[epoch] = losses.get(epoch, 0) + loss.item()
    print("Epoch: ", epoch, " Loss: ", losses[epoch])

C:\Users\thiru\AppData\Local\Temp\ipykernel_24500\2984377441.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:233.)
  X = torch.FloatTensor([doc.vector for doc in docs])


TypeError: len() of unsized object

In [32]:
# Train your model
for epoch in range(epochs):
    random.seed(epoch)
    losses = {}
    batches = minibatch(zip(texts, labels), size=compounding(batch_size, len(texts), 1.001))
    for batch in batches:
        texts, labels = zip(*batch)
        docs = [Doc(nlp.vocab, words=text.split(' ')) for text in texts]
        X = torch.FloatTensor([doc.vector for doc in docs])
        y = torch.FloatTensor([np.asarray(label) for label in labels])
        model.zero_grad()
        preds = model(X)
        loss = loss_fn(preds, y)
        loss.backward()
        optimizer.step()
        losses[epoch] = losses.get(epoch, 0) + loss.item()
    print("Epoch: ", epoch, " Loss: ", losses[epoch])


ValueError: [E031] Invalid token: empty string ('') at position 119.

In [11]:
# Test your model
text = "John lives in New York City."
words = text.split(' ')
doc = Doc(nlp.vocab, words=words)
X = torch.FloatTensor([doc.vector])
preds = model(X)
print(preds)


RuntimeError: input.size(-1) must be equal to input_size. Expected 300, got 0

In [ ]:
# Save your model
nlp.to_disk("my_model")